# 추출 과정에서 reference example 사용하는 법
- 추출의 성능은 LLM에 reference example을 제공해서 향상시킬 수 있다.
- 데이터 추출은 텍스트나 비정형 데이터 안의 정보를 구조화해서 생성한다.
- 이런 경우에 LLM의 Tool-calling 기능이 사용된다.
- 본 실습은 추출 및 유사한 어플리케이션 동작을 조정하는데 도움이 되는 도구 호출의 몇 가지 예를 작성하는 방법을 보여준다.
- Langchain은 tool-call 기능을 포함하는 LLM의 메시지에 tool-call 속성을 구현한다.
- 데이터 추출과정을 구현하기 위해서 `Human Message, AI Message, Tool Message` 순서의 chat history를 생성했다.

# Prompt Template

In [2]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
# 지시사항과 문맥(context)를 추가해서 Custom Prompt를 구성하세요
# 1) 예제들을 추가해서 Extraction 품질을 높일 수 있습니다.
# 2) 추가적인 매개변수를 통해서 문맥을 고려할 수 있습니다.
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attrivute asked "
            "to extract, return null for the attribute's value.",
        ),
        MessagesPlaceholder("examples"),# 이자리에 예제가 들어간다.
        ("human", "{text}"),
    ]
)

### Template Test

In [6]:
from langchain_core.messages import (
    HumanMessage,
)
prompt.invoke(
    {
        "text": "this is some text", "examples":[HumanMessage(content="testing 1 2 3")]
    }
)

ChatPromptValue(messages=[SystemMessage(content="You are an expert extraction algorithm. Only extract relevant information from the text. If you do not know the value of an attrivute asked to extract, return null for the attribute's value."), HumanMessage(content='testing 1 2 3'), HumanMessage(content='this is some text')])

# Define the schema

In [ ]:
from typing import List, Optional
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

class Person(BaseModel):
    """Information about a person."""
    # 위의 docstring은 LLM에 전달되어 Person이라는 스키마의 설명으로 사용되고 추출 성능을 향상하는데 도움이 될 수 있습니다.
    # 모든 필드는 Optional로 모델이 추출을 거부할 수 있습니다.
    # 각 필드는 description이 존재해서 LLM이 사용합니다.
    name: Optional[str] = Field(..., description="The name of the person")
    hair_color: Optional[str] = Field(
        ..., description = "The color of the person's hair if known"
    )
    height_in_meters: Optional[str] = Field(..., description="Height in METERs")

class Data(BaseModel):
    """Extracted data about people"""

    # model을 생성해서 여러 엔티티를 추출할 수 있게 한다.
    people: List[Person]

# Define reference examples